# PI

In [37]:
import csv
import re
from io import StringIO
#import fg_log_parser.py

#Setup

def split_kv(line):
    """
    Splits lines in key and value pairs and returns a dictionary.
    Example:
        >>> line = 'srcip=192.168.1.1 dstip=8.8.8.8 \
        ...         dport=53 proto=53 dstcountry="United States"'
        >>> split_kv(line)
        {'srcip': '192.168.1.1', 'dport': '53', 'dstip': '8.8.8.8', 'dstcountry': '"United States"', 'proto': '53'}
    """
    kvdelim = '='  # key and value deliminator
    logline = {}  # dictionary for logline
    # split line in key and value pairs
    # regex matches internal sub strings such as key = "word1 word2"
    for field in re.findall(r'(?:[^\s,""]|"(?:\\.|[^""])*")+', line):
        if kvdelim in field:
            key, value = field.split(kvdelim)
            logline[key] = value
    return logline

def get_communication_matrix(logfile,
                             logformat,
                             countbytes=False,
                             noipcheck=False,
                             showaction=False):
    """
    Reads firewall logfile and returns communication matrix as a dictionary.
    Parameters:
        logfile         Logfile to parse
        logformat       dictionary containing log format
        countbytes      sum up bytes sent and received
    Sample return matrix (one logline parsed):
        {'192.168.1.1': {'8.8.8.8': {'53': {'UDP': {'count': 1}}}}}
    Example:
    """

    log.info("get_communication_matrix() started with parameters: ")
    log.info("Option logfile: %s", logfile)
    log.info("Option countbytes: %s", countbytes)
    log.info("Option showaction: %s", showaction)

    # assign log format options from logformat dict
    srcipfield = logformat['srcipfield']
    dstipfield = logformat['dstipfield']
    dstportfield = logformat['dstportfield']
    protofield = logformat['protofield']
    sentbytesfield = logformat['sentbytesfield']
    rcvdbytesfield = logformat['rcvdbytesfield']
    actionfield = logformat['actionfield']

    matrix = {}  # communication matrix

    with open(logfile, 'r') as infile:
        # parse each line in file
        linecount = 1  # linecount for detailed error message

        for line in infile:
            """
            For loop creates a nested dictionary with multiple levels.
            Level description:
            Level 1:        srcips (source ips)
            Level 2:        dstips (destination ips)
            Level 3:        dstport (destination port number)
            Level 4:        proto (protocol number)
            Level 4.5:      action (Fortigate action)
            Level 5:        occurrence count
                            sentbytes
                            rcvdbytes
            """

            # check if necessary fields are in first line
            if linecount is 1 and not noipcheck:
                # print error message if srcip or dstip are missing
                if not check_log_format(line, srcipfield, dstipfield):
                    log.error("srcipfield or dstipfield not in line: %s ", linecount)
                    log.error("Check Log Format options and consult help message!")
                    sys.exit(1)

            # split each line in key and value pairs.
            logline = split_kv(line)
            linecount += 1

            # get() does substitute missing values with None
            # missing log fields will show None in the matrix
            srcip = logline.get(srcipfield)
            dstip = logline.get(dstipfield)
            dstport = logline.get(dstportfield)
            proto = translate_protonr(logline.get(protofield))
            # user has set --action
            if showaction:
                action = logline.get(actionfield)
            # if user has set --countbytes
            if countbytes:
                sentbytes = logline.get(sentbytesfield)
                rcvdbytes = logline.get(rcvdbytesfield)
            # extend matrix for each source ip
            if srcip not in matrix:
                log.info("Found new srcip %s", srcip)
                matrix[srcip] = {}
            # extend matrix for each dstip in srcip
            if dstip not in matrix[srcip]:
                log.info("Found new dstip: %s for sourceip: %s", dstip, srcip)
                matrix[srcip][dstip] = {}
            # extend matrix for each port in comm. pair
            if dstport not in matrix[srcip][dstip]:
                matrix[srcip][dstip][dstport] = {}
            # if proto not in matrix extend matrix
            if proto not in matrix[srcip][dstip][dstport]:
                matrix[srcip][dstip][dstport][proto] = {}
                matrix[srcip][dstip][dstport][proto]["count"] = 1
                if showaction:
                    matrix[srcip][dstip][dstport][proto]["action"] = action
                if countbytes:
                    matrix[srcip][dstip][dstport][proto]["sentbytes"] \
                        = int(sentbytes)
                    matrix[srcip][dstip][dstport][proto]["rcvdbytes"] \
                        = int(rcvdbytes)
            # if proto is already in matrix
            # increase count of variable count and sum bytes
            elif proto in matrix[srcip][dstip][dstport]:
                matrix[srcip][dstip][dstport][proto]["count"] += 1
                if countbytes:
                    try:
                        matrix[srcip][dstip][dstport][proto]["sentbytes"] \
                            += int(sentbytes)
                    except TypeError:
                        pass
                    try:
                        matrix[srcip][dstip][dstport][proto]["rcvdbytes"] \
                            += int(rcvdbytes)
                    except TypeError:
                        pass
        log.info("Parsed %s lines in logfile: %s ", linecount, logfile)
    return matrix


logformat = {'srcipfield': arguments['--srcipfield'],
                 'dstipfield': arguments['--dstipfield'],
                 'dstportfield': arguments['--dstportfield'],
                 'protofield': arguments['--protofield'],
                 'sentbytesfield': arguments['--sentbytesfield'],
                 'rcvdbytesfield': arguments['--rcvdbytesfield'],
                 'actionfield': arguments['--actionfield']
                 }

ModuleNotFoundError: No module named 'fg_log_parser'

In [34]:
with open('logs.log') as csvfile:
    data = csvfile.read()

In [35]:
data

'date=2018-11-28 time=18:31:12 bid=98608256 dvid=2112 itime=1543440672 logver=56 logid="0000000013" type="traffic" subtype="forward" level="notice" action="close" trandisp="noop" srcport=45884 dstport=5222 srcip=200.32.63.18 dstip=31.13.94.54 service="tcp/5222" proto=6 duration=604 policyid=1 sentbyte=2950 rcvdbyte=3785 sentpkt=39 rcvdpkt=38 srcintf="internal" dstintf="wan1" sessionid=16215098 app="WhatsApp" appcat="Collaboration" dstcountry="Argentina" applist="SOC_Standard" appid=28057 srccountry="Argentina" utmaction="allow" poluuid="78b8a352-0008-51e8-1b6d-44138f449f97" apprisk="elevated" countapp=1 policytype="policy" apps={WhatsApp} saasinfo={0} srcintfrole="lan" dstintfrole="wan" eventtime=1543440672 devid="FGT90D3Z16020197" vd="root" devname="REF1016064_OBRA_SOCIAL_DEL_PERS_CIVIL_DE_LA_NACI" utmref="BAQAAAAEAAAByTwRK7zUgNnjEBv9b"\ndate=2018-11-28 time=18:31:12 bid=98608253 dvid=2112 itime=1543440672 logver=56 logid="0000000020" type="traffic" subtype="forward" level="notice" ac

ValueError: too many values to unpack (expected 2)